In [ ]:
# 构建计算图-LSTM
#     embedding
#     LSTM
#     fc
#     train_op
# 训练流程代码
# 数据集封装
#     api: next_batch(batch_size)
# 词表封装:
#     api: sentenceToId(text_sentence):句子转换id
# 类别封装:
#     api: categoryToId(text_category).

In [1]:
import logging

import tensorflow.compat.v1 as tf
import os
import sys
import numpy as np
import math

tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.INFO)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class HParams:
    def __init__(self,
                 num_embedding_size,
                 num_timesteps,
                 num_lstm_nodes,
                 num_lstm_layers,
                 num_fc_nodes,
                 batch_size,
                 clip_lstm_grads,
                 learning_rate,
                 # 字符出现的阈值
                 num_word_threshold):
        self._num_embedding_size = num_embedding_size
        self._num_timesteps = num_timesteps
        self._num_lstm_nodes = num_lstm_nodes
        self._num_lstm_layers = num_lstm_layers
        self._num_fc_nodes = num_fc_nodes
        self._batch_size = batch_size
        self._clip_lstm_grads = clip_lstm_grads
        self._learning_rate = learning_rate
        # 字符出现的阈值
        self._num_word_threshold = num_word_threshold

    @property
    def num_embedding_size(self):
        return self._num_embedding_size
    @property
    def num_timesteps(self):
        return self._num_timesteps
    @property
    def num_lstm_nodes(self):
        return self._num_lstm_nodes
    @property
    def num_lstm_layers(self):
        return self._num_lstm_layers
    @property
    def num_fc_nodes(self):
        return self._num_fc_nodes
    @property
    def batch_size(self):
        return self._batch_size
    @property
    def clip_lstm_grads(self):
        return self._clip_lstm_grads
    @property
    def learning_rate(self):
        return self._learning_rate
    @property
    def num_word_threshold(self):
        return self._num_word_threshold

In [3]:
hps = HParams(
        num_embedding_size = 16,
        num_timesteps = 50,
        num_lstm_nodes = [32, 32],
        num_lstm_layers = 2,
        num_fc_nodes = 32,
        batch_size = 100,
        clip_lstm_grads = 1.0,
        learning_rate = 0.001,
        # 字符出现次数最小值
        num_word_threshold = 10)

In [4]:
train_file = 'cnews_data/cnews.train.seg.txt'
val_file = 'cnews_data/cnews.val.seg.txt'
test_file = 'cnews_data/cnews.test.seg.txt'
vocab_file = 'cnews_data/cnews.vocab.txt'
category_file = 'cnews_data/cnews.category.txt'
output_folder = 'cnews_data/run_text_rnn'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [5]:
class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)

    def _read_dict(self, filename):
        with open(filename, 'r', encoding = 'utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx

    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)

    @property
    def unk(self):
        return self._unk

    def size(self):
        return len(self._word_to_id)

    def sentence_to_id(self, sentence):
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids

class CategoryDict:
    def __init__(self, file_name):
        self._category_to_id = {}
        with open(file_name, 'r', encoding = 'utf-8') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx

    def size(self):
        return len(self._category_to_id)
    def category_to_id(self, category):
        if not category in self._category_to_id:
            raise Exception("%s is not in our category list" % category)
        return self._category_to_id[category]

vocab = Vocab(vocab_file, hps.num_word_threshold)
vocab_size = vocab.size()
tf.logging.info('vocab_size: %d' % vocab_size)
category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
tf.logging.info('category_size: %d' % num_classes)

INFO:tensorflow:vocab_size: 77331
INFO:tensorflow:category_size: 10


In [6]:
class TextDataSet:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix
        self._inputs = []
        # vector
        self._outputs = []
        self._indicator = 0
        self._parse_file(filename)

    def _parse_file(self,filename):
        tf.logging.info("Loading data from %s", filename)
        with open(filename, 'r', encoding = 'utf-8') as f:
            lines = f.readlines()
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            id_label = self._category_vocab.category_to_id(label)
            id_words = self._vocab.sentence_to_id(content)
            id_words = id_words[0:self._num_timesteps]
            padding_num = self._num_timesteps - len(id_words)
            id_words = id_words + \
                       [self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)
            self._outputs.append(id_label)
        self._inputs = np.asarray(self._inputs, dtype = np.int32)
        self._outputs = np.asarray(self._outputs, dtype = np.int32)
        self._random_shuffle()
        self._num_examples = len(self._inputs)

    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]

    def num_examples(self):
        return self._num_examples

    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):
            raise Exception("batch_size: %d is too large" % batch_size)

        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_outputs = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_outputs


In [7]:
train_dataset = TextDataSet(
    train_file, vocab, category_vocab, hps.num_timesteps
)
val_dataset = TextDataSet(
    val_file, vocab, category_vocab, hps.num_timesteps
)
test_dataset = TextDataSet(
    test_file, vocab, category_vocab, hps.num_timesteps
)
print(train_dataset.next_batch(2))
print(val_dataset.next_batch(2))
print(test_dataset.next_batch(2))

INFO:tensorflow:Loading data from cnews_data/cnews.train.seg.txt
INFO:tensorflow:Loading data from cnews_data/cnews.val.seg.txt
INFO:tensorflow:Loading data from cnews_data/cnews.test.seg.txt
(array([[ 2173,   162,  6076,  7020,    37,     0,   374,    17,   250,
           16,   976,   149,     2,    30, 46755,     1,    18,     4,
        28096, 10342, 23911,     2,     0,  1465,     1,  1291,    30,
         2134,     2, 70262,  5633,     3,    60,  5633,  4753,   181,
            0,     2,     1,    19,  6758,  1069,    18,   126, 41083,
            3,  4595,    19,   449,    18],
       [ 9801,   128,   502,   433,   650,    61,   685,   197,    17,
          250,    16, 10815,  9801,   267,  1624,  7134,   157,  2385,
          385,     1,  3523,  9801,   926,  4936,     0,   377,    84,
            1,  2239,     1,   544, 21729,    12,  1330,   173,    37,
           77, 13692,     6,  4828,     3,   267,   459,   730,  4574,
          358,   220,   375,     1, 28281]]), array([

In [15]:
def create_model(hps, vocab_size, num_classes):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size

    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    # dropout 保存下来的值
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    # 保存模型训练到哪一步

    global_step = tf.Variable(
        tf.zeros([], tf.int64), name = 'global_step', trainable = False
    )
    # 随机的均匀分布中初始化
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embedding', initializer = embedding_initializer):
         embeddings = tf.get_variable(
             'embeddinggg',
             [vocab_size, hps.num_embedding_size],
             tf.float32
         )
         # 对应inputs & embeddings 拼接成矩阵
         # [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]]
         embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)

    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_lstm_nodes[-1]) / 3.0
    lstm_init = tf.random_uniform_initializer(-scale, scale)

    def _generate_params_for_lstm_cell(x_size, h_size, bias_size):
        """
        generates parameters for pure lstm implementation.
        :param X_size: 输入大小
        :param h_size: 上一步输出大小
        :param bias_size:
        :return:
        """
        x_w = tf.get_variable('x_weights', x_size)
        h_w = tf.get_variable('h_weights', h_size)
        b = tf.get_variable('biases', bias_size,
                            initializer = tf.constant_initializer(0.0))
        return x_w, h_w, b

    with tf.variable_scope('lstm_nn', initializer = lstm_init):
        """
        cells = []
        for i in range(hps.num_lstm_layers):
            cell = tf.nn.rnn_cell.BasicLSTMCell(
                hps.num_lstm_nodes[i],
                state_is_tuple = True
            )
            cell = tf.nn.rnn_cell.DropoutWrapper(
                cell,
                output_keep_prob = keep_prob
            )
            cells.append(cell)
        cell = tf.nn.rnn_cell.MultiRNNCell(cells)

        initial_state = cell.zero_state(batch_size, tf.float32)
        #rnn_outputs: [batch_size, num_timesteps, lstm_outputs[-1]]
        rnn_outputs, _ = tf.nn.dynamic_rnn(
            cell,embed_inputs,initial_state = initial_state)
        last = rnn_outputs[:,-1,:]
        """
        with tf.variable_scope('inputs'):
            ix, ih, ib = _generate_params_for_lstm_cell(
                x_size = [hps.num_embedding_size, hps.num_lstm_nodes[0]],
                h_size = [hps.num_lstm_nodes[0], hps.num_lstm_nodes[0]],
                bias_size = [1, hps.num_lstm_nodes[0]]
            )
        with tf.variable_scope('outputs'):
            ox, oh, ob = _generate_params_for_lstm_cell(
                x_size = [hps.num_embedding_size, hps.num_lstm_nodes[0]],
                h_size = [hps.num_lstm_nodes[0], hps.num_lstm_nodes[0]],
                bias_size = [1, hps.num_lstm_nodes[0]]
            )
        with tf.variable_scope('forget'):
            fx, fh, fb = _generate_params_for_lstm_cell(
                x_size = [hps.num_embedding_size, hps.num_lstm_nodes[0]],
                h_size = [hps.num_lstm_nodes[0], hps.num_lstm_nodes[0]],
                bias_size = [1, hps.num_lstm_nodes[0]]
            )
        with tf.variable_scope('memory'):
            cx, ch, cb = _generate_params_for_lstm_cell(
                x_size = [hps.num_embedding_size, hps.num_lstm_nodes[0]],
                h_size = [hps.num_lstm_nodes[0], hps.num_lstm_nodes[0]],
                bias_size = [1, hps.num_lstm_nodes[0]]
            )
        state = tf.Variable(
            tf.zeros([batch_size, hps.num_lstm_nodes[0]]),
            trainable = False
        )
        h = tf.Variable(
            tf.zeros([batch_size, hps.num_lstm_nodes[0]]),
            trainable = False
        )

        for i in range(num_timesteps):
            # [batch_size, 1, embed_size]
            embed_input = embed_inputs[:,i,:]
            embed_input = tf.reshape(embed_input,
                                      [batch_size, hps.num_embedding_size])
            forget_gate = tf.sigmoid(
                tf.matmul(embed_input, fx) + tf.matmul(h, fh) + fb)
            input_gate = tf.sigmoid(
                tf.matmul(embed_input, ix) + tf.matmul(h, ih) + ib)
            output_gate = tf.sigmoid(
                tf.matmul(embed_input, ox) + tf.matmul(h, oh) + ob)
            mid_state = tf.tanh(
                tf.matmul(embed_input, cx) + tf.matmul(h, ch) + cb)
            state = mid_state * input_gate + state * forget_gate
            h = output_gate * tf.tanh(state)
        last = h


    fc_init = tf.uniform_unit_scaling_initializer(factor = 1.0)
    with tf.variable_scope('fc', initializer = fc_init):
        fc1 = tf.layers.dense(last,
                              hps.num_fc_nodes,
                              activation = tf.nn.relu,
                              name = 'fc1')
        fc1_dropout = tf.layers.dropout(fc1, keep_prob)
        # 映射到类别
        logits = tf.layers.dense(fc1_dropout, num_classes, name = 'fc2')

    with tf.name_scope('metrics'):
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = logits, labels = outputs
        )
        loss = tf.reduce_mean(softmax_loss)
        # [0,2,5,4,2] -> argmax: 2 在2的维度上最大
        y_pred = tf.argmax(tf.nn.softmax(logits),
                           1,
                           output_type = tf.int32)
        correct_pred = tf.equal(outputs, y_pred)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    with tf.name_scope('train_op'):
        tvars = tf.trainable_variables()
        for var in tvars:
            tf.logging.info('variable name %s' % var.name)
        # 限制梯度大小
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), hps.clip_lstm_grads)
        optimizer = tf.train.AdamOptimizer(hps.learning_rate)
        train_op = optimizer.apply_gradients(zip(grads, tvars),
                                             global_step = global_step)

        return ((inputs, outputs, keep_prob),
                (loss, accuracy),
                (train_op, global_step))


In [16]:
tf.reset_default_graph()
placeholders, metrics, others = create_model(
    hps, vocab_size, num_classes)

inputs, outputs, keep_prob = placeholders
loss, accuracy = metrics
train_op, global_step = others

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
INFO:tensorflow:variable name embedding/embeddinggg:0
INFO:tensorflow:variable name lstm_nn/inputs/x_weights:0
INFO:tensorflow:variable name lstm_nn/inputs/h_weights:0
INFO:tensorflow:variable name lstm_nn/inputs/biases:0
INFO:tensorflow:variable name lstm_nn/outputs/x_weights:0
INFO:tensorflow:variable name lstm_nn/outputs/h_weights:0
INFO:tensorflow:variable name lstm_nn/outputs/biases:0
INFO:tensorflow:variable name lstm_nn/forget/x_weights:0
INFO:tensorflow:variable name lstm_nn/forget/h_weights:0
INFO:tensorflow:variable name lstm_nn/forget/biases:0
INFO:tensorflow:variable name lstm_nn/memory/x_weights:0
INFO:tensorflow:variable name lstm_nn/memory/h_weights:0
INFO:ten

In [17]:
def eval_holdout(sess, accuracy, dataset_for_test, batch_size):
    num_batches = dataset_for_test.num_examples() // batch_size
    tf.logging.info("Eval holdout: num_examples = %d, batch_size = %d",
                    dataset_for_test.num_examples(), batch_size)
    accuracy_vals = []
    for i in range(num_batches):
        batch_inputs, batch_labels = dataset_for_test.next_batch(batch_size)
        accuracy_val = sess.run(accuracy,
                                feed_dict = {
                                    inputs: batch_inputs,
                                    outputs: batch_labels,
                                    keep_prob: 1.0,
                                })
        accuracy_vals.append(accuracy_val)
    return np.mean(accuracy_vals)

In [18]:
init_op = tf.global_variables_initializer()
train_keep_prob_value = 0.8
test_keeo_prob_value = 1.0

num_train_steps = 10000

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        batch_inputs,batch_labels = train_dataset.next_batch(hps.batch_size)
        outputs_val = sess.run([loss, accuracy,train_op, global_step],
                           feed_dict = {
                               inputs: batch_inputs,
                               outputs: batch_labels,
                               keep_prob: train_keep_prob_value
                           })
        loss_val, accuracy_val, _,global_step_val = outputs_val
        if global_step_val % 200 == 0:
            tf.logging.info("Step: %5d, loss: %3.3f, accuracy: %3.3f"
                            % (global_step_val, loss_val, accuracy_val))
        if global_step_val % 1000 == 0:
            accuracy_eval = eval_holdout(sess, accuracy, val_dataset, hps.batch_size)
            accuracy_test = eval_holdout(sess, accuracy, test_dataset, hps.batch_size)
            tf.logging.info("Step: %5d, val_accuracy: %3.3f, test_accuracy: %3.3f"
                            % (global_step_val, accuracy_eval, accuracy_test))


INFO:tensorflow:Step:   200, loss: 2.072, accuracy: 0.190
INFO:tensorflow:Step:   400, loss: 1.534, accuracy: 0.360
INFO:tensorflow:Step:   600, loss: 1.503, accuracy: 0.450
INFO:tensorflow:Step:   800, loss: 1.227, accuracy: 0.510
INFO:tensorflow:Step:  1000, loss: 1.074, accuracy: 0.610
INFO:tensorflow:Eval holdout: num_examples = 5000, batch_size = 100
INFO:tensorflow:Eval holdout: num_examples = 10000, batch_size = 100
INFO:tensorflow:Step:  1000, val_accuracy: 0.505, test_accuracy: 0.544
INFO:tensorflow:Step:  1200, loss: 0.813, accuracy: 0.670
INFO:tensorflow:Step:  1400, loss: 0.818, accuracy: 0.690
INFO:tensorflow:Step:  1600, loss: 0.868, accuracy: 0.720
INFO:tensorflow:Step:  1800, loss: 0.809, accuracy: 0.800
INFO:tensorflow:Step:  2000, loss: 0.596, accuracy: 0.770
INFO:tensorflow:Eval holdout: num_examples = 5000, batch_size = 100
INFO:tensorflow:Eval holdout: num_examples = 10000, batch_size = 100
INFO:tensorflow:Step:  2000, val_accuracy: 0.666, test_accuracy: 0.696
INFO